In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import rand

In [ ]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("Feature Extraction and Transformation using Spark").getOrCreate()

# Tokenizer - is used to break a sentence into words

In [ ]:
#import tokenizer
from pyspark.ml.feature import Tokenizer

In [ ]:
#create a sample dataframe
sentenceDataFrame = spark.createDataFrame([
    (1, "Spark is a distributed computing system."),
    (2, "It provides interfaces for multiple languages"),
    (3, "Spark is built on top of Hadoop")
], ["id", "sentence"])

In [ ]:
#display the dataframe
sentenceDataFrame.show(truncate = False)

In [ ]:
#create tokenizer instance.
#mention the column to be tokenized as inputcol
#mention the output column name where the tokens are to be stored.
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [ ]:
#tokenize
token_df = tokenizer.transform(sentenceDataFrame)

In [ ]:
#display the tokenized data
token_df.show(truncate=False)

# CountVectorizer -  convert text into numerical format. It gives the count of each word in a given document.

In [ ]:
#import CountVectorizer
from pyspark.ml.feature import CountVectorizer

In [ ]:
#create a sample dataframe and display it.
textdata = [(1, "I love Spark Spark provides Python API ".split()),
            (2, "I love Python Spark supports Python".split()),
            (3, "Spark solves the big problem of big data".split())]

textdata = spark.createDataFrame(textdata, ["id", "words"])

textdata.show(truncate=False)

In [ ]:
# Create a CountVectorizer object
# mention the column to be count vectorized as inputcol
# mention the output column name where the count vectors are to be stored.
cv = CountVectorizer(inputCol="words", outputCol="features")

In [ ]:
# Fit the CountVectorizer model on the input data
model = cv.fit(textdata)

In [ ]:
# Transform the input data to bag-of-words vectors
result = model.transform(textdata)

In [ ]:
# display the dataframe
result.show(truncate=False)

# TF - IDF

## Term Frequency-Inverse Document Frequency is used to quantify the importance of a word in a document. TF-IDF is computed by multiplying the number of times a word occurs in a document by the inverse document frequency of the word.

In [ ]:
#import necessary classes for TF-IDF calculation
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


In [ ]:
#create a sample dataframe and display it.
sentenceData = spark.createDataFrame([
        (1, "Spark supports python"),
        (2, "Spark is fast"),
        (3, "Spark is easy")
    ], ["id", "sentence"])

sentenceData.show(truncate = False)

In [ ]:
#tokenize the "sentence" column and store in the column "words"
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show(truncate = False)

In [ ]:
# Create a HashingTF object
# mention the "words" column as input
# mention the "rawFeatures" column as output

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10)
featurizedData = hashingTF.transform(wordsData)

featurizedData.show(truncate = False)

In [ ]:
# Create an IDF object
# mention the "rawFeatures" column as input
# mention the "features" column as output

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tfidfData = idfModel.transform(featurizedData)

In [ ]:
#display the tf-idf data
tfidfData.select("sentence", "features").show(truncate=False)

# StopWordsRemover 
### StopWordsRemover is a transformer that filters out stop words like "a","an" and "the"

In [ ]:
#import StopWordsRemover
from pyspark.ml.feature import StopWordsRemover

In [ ]:
#create a dataframe with sample text and display it
textData = spark.createDataFrame([
    (1, ['Spark', 'is', 'an', 'open-source', 'distributed', 'computing', 'system']),
    (2, ['IT', 'has', 'interfaces', 'for', 'multiple', 'languages']),
    (3, ['It', 'has', 'a', 'wide', 'range', 'of', 'libraries', 'and', 'APIs'])
], ["id", "sentence"])

textData.show(truncate = False)

In [ ]:
# remove stopwords from "sentence" column and store the result in "filtered_sentence" column
remover = StopWordsRemover(inputCol="sentence", outputCol="filtered_sentence")
textData = remover.transform(textData)

In [ ]:
# display the dataframe
textData.show(truncate = False)

# StringIndexer

In [ ]:
#import StringIndexer
from pyspark.ml.feature import StringIndexer

In [ ]:
#create a dataframe with sample text and display it
colors = spark.createDataFrame(
    [(0, "red"), (1, "red"), (2, "blue"), (3, "yellow" ), (4, "yellow"), (5, "yellow")],
    ["id", "color"])

colors.show()

In [ ]:
# index the strings in the column "color" and store their indexes in the column "colorIndex"
indexer = StringIndexer(inputCol="color", outputCol="colorIndex")
indexed = indexer.fit(colors).transform(colors)

In [ ]:
# display the dataframe
indexed.show()

# StandardScaler

### StandardScaler transforms the data so that it has a mean of 0 and a standard deviation of 1

In [ ]:
#import StandardScaler
from pyspark.ml.feature import StandardScaler


In [ ]:
# Create a sample dataframe and display it
from pyspark.ml.linalg import Vectors
data = [(1, Vectors.dense([70, 170, 17])),
        (2, Vectors.dense([80, 165, 25])),
        (3, Vectors.dense([65, 150, 135]))]
df = spark.createDataFrame(data, ["id", "features"])

df.show()

In [ ]:
# Define the StandardScaler transformer
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

In [ ]:
# Fit the transformer to the dataset
scalerModel = scaler.fit(df)

In [ ]:
# Scale the data
scaledData = scalerModel.transform(df)

In [ ]:
# Show the scaled data
scaledData.show(truncate = False)

In [ ]:
spark.stop()